<a href="https://colab.research.google.com/github/w-okada/voice-changer/blob/dev/VoiceChangerDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MMVCプレイヤー（普通版）
---

このノートはColab上でMMVCのボイチェンを行うノートです。

正式版はローカルPCのDocker上で動かすアプリケーションです。

正式版は、多くの場合より少ないタイムラグで滑らかに音声を変換できます。

詳細な使用方法はこちらの[リポジトリ](https://github.com/w-okada/voice-changer)からご確認ください。


# GPUを確認
GPUを用いたほうが高速に処理が行えます。

下記のコマンドでGPUが確認できない場合は、上のメニューから

「ランタイム」→「ランタイムの変更」→「ハードウェア アクセラレータ」

でGPUを選択してください。

In [1]:
# (1) GPUの確認
!nvidia-smi

Sun Jan 29 12:09:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 使用するモデルとコンフィグファイルの指定

使用するトレーニング済みのモデルと、トレーニングで使用したコンフィグファイルのパスを指定してください。

多くの場合はGoogle Driveに格納されているファイルを使用すると思います。その場合は、下の(2-2)のセルを実行してドライブをマウントしてください

In [2]:
# (2-1) 使用するモデルとコンフィグファイルの指定
if "MODEL" in locals():
  del MODEL
if "ONNX" in locals():
  del ONNX

CONFIG="/content/drive/MyDrive/VoiceChanger/config.json"
#MODEL="/content/drive/MyDrive/VoiceChanger/G_326000.pth"
ONNX="/content/drive/MyDrive/VoiceChanger/G_326000.onnx"

In [3]:
# (2-2) Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# リポジトリのクローン
リポジトリをクローンします

In [4]:
# (3) リポジトリのクローン
!git clone --depth 1 https://github.com/w-okada/voice-changer.git  -b v.1.3.7
%cd voice-changer/server
!git clone https://github.com/isletennos/MMVC_Client.git
!cd MMVC_Client && git checkout 04f3fec4fd82dea6657026ec4e1cd80fb29a415c && cd -

Cloning into 'voice-changer'...
remote: Enumerating objects: 157, done.
remote: Counting objects: 100% (157/157), done.
remote: Compressing objects: 100% (141/141), done.
remote: Total 157 (delta 21), reused 70 (delta 5), pack-reused 0
Receiving objects: 100% (157/157), 1.61 MiB | 4.03 MiB/s, done.
Resolving deltas: 100% (21/21), done.
/content/voice-changer/server
Cloning into 'MMVC_Client'...
remote: Enumerating objects: 611, done.
remote: Counting objects: 100% (337/337), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 611 (delta 238), reused 285 (delta 214), pack-reused 274
Receiving objects: 100% (611/611), 752.38 KiB | 21.50 MiB/s, done.
Resolving deltas: 100% (360/360), done.
Note: switching to '04f3fec4fd82dea6657026ec4e1cd80fb29a415c'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If yo

# モジュールのインストール

必要なモジュールをインストールします。

In [5]:
# (5) 設定ファイルの確認
!apt-get install -y libsndfile1-dev &> /dev/null
!pip install fastapi &> /dev/null
!pip install pyOpenSSL &> /dev/null
!pip install python-multipart &> /dev/null
!pip install python-socketio &> /dev/null
!pip install uvicorn &> /dev/null
!pip install websockets &> /dev/null
!pip install onnxruntime-gpu &> /dev/null

# サーバの起動

サーバを起動します。(6-1)

サーバの起動状況を確認します。(6-2) 

このセルは繰り返し実行することになるのでCtrl+Retでセルを実行してください。

アクセスできるようになるまで、１～２分かかるようです。コーヒーでも飲みに行きましょう。

下記のようなテキストが表示されたら起動完了です。
(warningは無視して問題ありません。)
```
/usr/local/lib/python3.8/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'OpenVINOExecutionProvider' is not in available provider names.Available providers: 'TensorrtExecutionProvider, CUDAExecutionProvider, CPUExecutionProvider'
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'DmlExecutionProvider' is not in available provider names.Available providers: 'TensorrtExecutionProvider, CUDAExecutionProvider, CPUExecutionProvider'
  warnings.warn(
VoiceChanger Initialized (GPU_NUM:1, mps_enabled:False)
    Voice Changerを起動しています。
    -- 設定 -- 
    CONFIG:/content/drive/MyDrive/VoiceChanger/config.json, MODEL:None ONNX_MODEL:/content/drive/MyDrive/VoiceChanger/G_326000.onnx```



In [6]:
# (6-1) サーバの起動
import random
PORT = 10000 + random.randint(1, 9999)
LOG_FILE = f"LOG_FILE_{PORT}"

if "MODEL" in locals() and "ONNX" in locals():
  model_param = f" -m {MODEL} -o {ONNX}"
elif "MODEL" in locals():
  model_param = f" -m {MODEL}"
elif "ONNX" in locals():
  model_param = f" -o {ONNX}"
else:
  model_param = f""

get_ipython().system_raw(f'python3 MMVCServerSIO.py -t MMVC -p {PORT} -c {CONFIG} {model_param}  --https False --colab True >{LOG_FILE} 2>&1 &')
#print(f"PORT:{PORT}, LOG_FILE:{LOG_FILE}")

In [12]:
# (6-2) サーバの起動確認 (Ctrl+Retで実行)
!tail -20 {LOG_FILE}

/usr/local/lib/python3.8/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'OpenVINOExecutionProvider' is not in available provider names.Available providers: 'TensorrtExecutionProvider, CUDAExecutionProvider, CPUExecutionProvider'
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'DmlExecutionProvider' is not in available provider names.Available providers: 'TensorrtExecutionProvider, CUDAExecutionProvider, CPUExecutionProvider'
  warnings.warn(
VoiceChanger Initialized (GPU_NUM:1, mps_enabled:False)
    Voice Changerを起動しています。
    -- 設定 -- 
    CONFIG:/content/drive/MyDrive/VoiceChanger/config.json, MODEL:None ONNX_MODEL:/content/drive/MyDrive/VoiceChanger/G_326000.onnx


# プロキシを起動
ウェブサーバへのアクセスをするためのプロキシを起動します。

表示されたURLをクリックして開くと別タブでアプリが開きます。

Colabなので、ロードにある程度時間がかかります(30秒くらい)。

In [13]:
# (7) プロキシを起動
from google.colab.output import eval_js
proxy = eval_js( "google.colab.kernel.proxyPort(" + str(PORT) + ")" )
print(f"{proxy}front/?colab=true")

https://nafdn2dmorn-496ff2e9c6d22116-18341-colab.googleusercontent.com/front/?colab=true
